In [1]:
import gpflow
import tensorflow as tf
from gpflow.kernels.stationaries import SquaredExponential, Kernel, Matern52
from tensorflow import Tensor
import numpy as np

2023-06-17 14:17:36.473023: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 14:17:36.594846: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 14:17:36.595543: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 14:17:37.332206: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# old stuff

def old_k_grad_se(inducing_variable_ny, inducing_variable_Zy, kernel):
    funcs2 = tf.range(inducing_variable_Zy.shape[0])
    norm = 1 / (kernel.lengthscales ** 2)
    partial_derivative = tf.map_fn(
        lambda j: (inducing_variable_ny - inducing_variable_Zy[j]),
        funcs2, fn_output_signature=tf.float64, parallel_iterations=8)
    return norm * partial_derivative

def old_kernel_derivative(
        inducing_location_ny: Tensor,
        inducing_location_Zy: Tensor,
        kernel: Kernel,
):
    iterator = tf.range(inducing_location_ny.shape[0])
    block = tf.map_fn(lambda i: old_k_grad_se(inducing_location_ny[i], inducing_location_Zy, kernel),
                      iterator, fn_output_signature=tf.float64, parallel_iterations=8)

    return block

def initialize_Z(num_latent_gps, num_inducing):
    Z = tf.convert_to_tensor(np.array(
        [np.full(num_latent_gps, i) for i in np.linspace(0.1, 0.9, num_inducing)], dtype=np.float64))
    return Z

In [3]:
dof = 1
from gpflow.base import default_float

start = tf.cast((tf.fill((1, dof), 0.)), dtype=default_float())
end = tf.cast((tf.fill((1, dof), 1.)), dtype=default_float())
Z = initialize_Z(dof, 2)
ny = tf.concat([start, end], axis=0)
Zy = tf.concat([ny, Z], axis=0)

2023-06-17 14:17:38.531675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-17 14:17:38.608651: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
Zy.shape

TensorShape([4, 1])

In [5]:
old_kernel_derivative(ny, Zy, SquaredExponential())

<tf.Tensor: shape=(2, 4, 1), dtype=float64, numpy=
array([[[ 0. ],
        [-1. ],
        [-0.1],
        [-0.9]],

       [[ 1. ],
        [ 0. ],
        [ 0.9],
        [ 0.1]]])>

In [6]:
def new_kernel_derivative(ny, Zy, kernel):
    lengthscale = kernel.lengthscales
    block = new_k_grad_se(ny, Zy, lengthscale)
    return block

def new_k_grad_se(ny, Zy, lengthscale):
    norm = 1 / (lengthscale ** 2)
    partial_derivative = norm * tf.subtract(tf.expand_dims(ny, 1), Zy)
    return partial_derivative


In [7]:
new_kernel_derivative(ny, Zy, SquaredExponential())

<tf.Tensor: shape=(2, 4, 1), dtype=float64, numpy=
array([[[ 0. ],
        [-1. ],
        [-0.1],
        [-0.9]],

       [[ 1. ],
        [ 0. ],
        [ 0.9],
        [ 0.1]]])>

In [8]:
new_kernel_derivative(Zy, Zy, SquaredExponential())

<tf.Tensor: shape=(4, 4, 1), dtype=float64, numpy=
array([[[ 0. ],
        [-1. ],
        [-0.1],
        [-0.9]],

       [[ 1. ],
        [ 0. ],
        [ 0.9],
        [ 0.1]],

       [[ 0.1],
        [-0.9],
        [ 0. ],
        [-0.8]],

       [[ 0.9],
        [-0.1],
        [ 0.8],
        [ 0. ]]])>

In [9]:
def k_grad_grad_se_fallback(
        inducing_location_ny: Tensor,
        kernel: SquaredExponential,
):
    norm = kernel.lengthscales ** 2
    inducing_diff = tf.expand_dims(inducing_location_ny, 1) - tf.expand_dims(inducing_location_ny, 0)
    second_derivative = norm - inducing_diff ** 2
    return second_derivative / kernel.lengthscales ** 4

In [10]:

@tf.function
def kernel_derivative(ny, Zy, kernel):

    block_original = kernel(Zy)[..., None]
    print(block_original.shape)
    ny_block = k_grad_grad_se_fallback(ny, kernel) * block_original[:ny.shape[0], :ny.shape[0]]
    print(ny_block.shape)
    nyZy_block = new_kernel_derivative(ny, Zy[ny.shape[0]:], kernel) * block_original[:ny.shape[0], ny.shape[0]:]
    print(nyZy_block.shape)
    upper_block = tf.concat([ny_block, nyZy_block], axis=1)
    lower_block = tf.concat([-tf.transpose(nyZy_block, perm=(1, 0, 2)), block_original[ny.shape[0]:, ny.shape[0]:]], axis=1)
    return tf.concat([upper_block, lower_block], axis=0)

In [11]:
aux = kernel_derivative(ny, Zy, SquaredExponential(lengthscales=.5))

(4, 4, 1)
(2, 2, 1)
(2, 2, 1)


In [12]:
tf.transpose(aux, perm=(2, 0 ,1))

<tf.Tensor: shape=(1, 4, 4), dtype=float64, numpy=
array([[[ 4.        , -1.6240234 , -0.39207947, -0.71243532],
        [-1.6240234 ,  4.        ,  0.71243532,  0.39207947],
        [ 0.39207947, -0.71243532,  1.        ,  0.2780373 ],
        [ 0.71243532, -0.39207947,  0.2780373 ,  1.        ]]])>

In [13]:
kk = SquaredExponential(lengthscales=.5)
# tf.print(kk)

In [14]:
ny

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[0.],
       [1.]])>

In [15]:
(ny[0] - ny[1])/(0.5 ** 2) * kk(ny) 

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[-4.        , -0.54134113],
       [-0.54134113, -4.        ]])>

In [16]:
new_kernel_derivative(ny, ny, kk) * kk(ny)

<tf.Tensor: shape=(2, 2, 2), dtype=float64, numpy=
array([[[ 0.        ,  0.        ],
        [-0.54134113, -4.        ]],

       [[ 4.        ,  0.54134113],
        [ 0.        ,  0.        ]]])>

In [ ]:
K = [0.062501 0.05859475 1e-06 -0.062499 -0.006249 -0.0084229130434782615 -0.010596826086956525 -0.012770739130434786 -0.014944652173913045 -0.0171185652173913 -0.019292478260869564 -0.02146639130434783 -0.023640304347826088 -0.02581421739130435 -0.027988130434782606 -0.030162043478260864 -0.03233595652173913 -0.034509869565217392 -0.036683782608695654 -0.03885769565217391 -0.041031608695652172 -0.043205521739130434 -0.045379434782608696 -0.047553347826086952 -0.049727260869565214 -0.051901173913043476 -0.054075086956521738 -0.056249]
  [0.05859475 0.062501 0.062501 1e-06 0.056251 0.05407708695652174 0.051903173913043478 0.049729260869565216 0.047555347826086961 0.0453814347826087 0.043207521739130436 0.041033608695652174 0.038859695652173912 0.03668578260869565 0.034511869565217394 0.032337956521739139 0.03016404347826087 0.027990130434782608 0.025816217391304346 0.02364230434782609 0.021468391304347828 0.019294478260869566 0.017120565217391304 0.014946652173913047 0.012772739130434784 0.010598826086956522 0.00842491304347826 0.0062509999999999987]
  [1e-06 -0.062499 0.500001 0.40176228684453036 0.49890844541724522 0.49801799801789431 0.49686607930323784 0.49545451174916094 0.49378552477394472 0.49186174887081535 0.489686208699424 0.48726231516165164 0.48459385649079978 0.48168498838674756 0.478540223233031 0.4751644184350019 0.47156276392124319 0.46774076885324445 0.46370424759095413 0.45945930496422421 0.45501232090233013 0.45036993447567875 0.44553902740550355 0.44052670709878128 0.43534028926678386 0.42998728018660409 0.4244753586656535 0.41881235776953657]
  [0.062501 1e-06 0.40176228684453036 0.500001 0.41881235776953657 0.4244753586656535 0.42998728018660409 0.43534028926678381 0.44052670709878128 0.44553902740550355 0.45036993447567875 0.45501232090233013 0.45945930496422421 0.46370424759095413 0.46774076885324445 0.47156276392124319 0.47516441843500185 0.47854022323303108 0.48168498838674756 0.48459385649079983 0.48726231516165164 0.48968620869942392 0.49186174887081541 0.49378552477394472 0.495454511749161 0.49686607930323784 0.49801799801789431 0.49890844541724522]
  [0.0062510000000000005 -0.056249 0.49890844541724522 0.41881235776953657 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924 0.48130469701492223 0.47813077932351289 0.47472645280913361 0.47109694804702379 0.4672478131307139 0.46318489954803271 0.45891434736941544 0.45444256980095132 0.44977623715650628 0.44492226030491427 0.43988777364963222 0.4346801176994029]
  [0.00842491304347826 -0.054075086956521738 0.49801799801789431 0.4244753586656535 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924 0.48130469701492223 0.47813077932351289 0.47472645280913361 0.47109694804702379 0.4672478131307139 0.46318489954803266 0.45891434736941544 0.45444256980095132 0.44977623715650628 0.44492226030491427 0.43988777364963222]
  [0.010598826086956524 -0.051901173913043476 0.49686607930323784 0.42998728018660409 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238352 0.48694205074937669 0.48424330621240924 0.48130469701492223 0.47813077932351289 0.47472645280913361 0.47109694804702379 0.4672478131307139 0.46318489954803271 0.45891434736941544 0.45444256980095132 0.44977623715650628 0.44492226030491427]
  [0.012772739130434784 -0.049727260869565214 0.49545451174916094 0.43534028926678381 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924 0.48130469701492229 0.47813077932351289 0.47472645280913361 0.47109694804702379 0.4672478131307139 0.46318489954803271 0.45891434736941544 0.45444256980095132 0.44977623715650628]
  [0.014946652173913043 -0.047553347826086959 0.49378552477394472 0.44052670709878128 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924 0.48130469701492223 0.47813077932351289 0.47472645280913361 0.47109694804702379 0.4672478131307139 0.46318489954803271 0.45891434736941544 0.45444256980095132]
  [0.017120565217391304 -0.045379434782608696 0.49186174887081535 0.44553902740550355 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924 0.48130469701492223 0.47813077932351283 0.47472645280913361 0.47109694804702379 0.4672478131307139 0.46318489954803266 0.45891434736941544]
  [0.019294478260869566 -0.043205521739130434 0.489686208699424 0.45036993447567875 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924 0.48130469701492223 0.47813077932351289 0.47472645280913361 0.47109694804702379 0.46724781313071395 0.46318489954803271]
  [0.021468391304347832 -0.041031608695652172 0.48726231516165164 0.45501232090233013 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238352 0.48694205074937669 0.48424330621240924 0.48130469701492229 0.47813077932351289 0.47472645280913361 0.47109694804702379 0.46724781313071395]
  [0.02364230434782609 -0.03885769565217391 0.48459385649079978 0.45945930496422421 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238352 0.48694205074937669 0.48424330621240924 0.48130469701492223 0.47813077932351289 0.47472645280913361 0.47109694804702379]
  [0.025816217391304352 -0.036683782608695648 0.48168498838674756 0.46370424759095413 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924 0.48130469701492229 0.47813077932351289 0.47472645280913361]
  [0.027990130434782608 -0.034509869565217392 0.478540223233031 0.46774076885324445 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553131 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924 0.48130469701492223 0.47813077932351283]
  [0.030164043478260866 -0.032335956521739137 0.4751644184350019 0.47156276392124319 0.48424330621240924 0.48694205074937669 0.48939672973238352 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924 0.48130469701492223]
  [0.032337956521739132 -0.030162043478260868 0.47156276392124319 0.47516441843500185 0.48130469701492223 0.48424330621240924 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669 0.48424330621240924]
  [0.034511869565217394 -0.027988130434782606 0.46774076885324445 0.47854022323303108 0.47813077932351289 0.48130469701492223 0.48424330621240924 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238346 0.48694205074937669]
  [0.036685782608695656 -0.025814217391304344 0.46370424759095413 0.48168498838674756 0.47472645280913361 0.47813077932351289 0.48130469701492223 0.48424330621240924 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221 0.49160350843613565 0.48939672973238352]
  [0.038859695652173912 -0.023640304347826088 0.45945930496422421 0.48459385649079983 0.47109694804702379 0.47472645280913361 0.47813077932351289 0.48130469701492229 0.48424330621240924 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553131 0.4935589282851221 0.49160350843613565]
  [0.041033608695652174 -0.021466391304347826 0.45501232090233013 0.48726231516165164 0.4672478131307139 0.47109694804702379 0.47472645280913361 0.47813077932351289 0.48130469701492223 0.48424330621240924 0.48694205074937669 0.48939672973238352 0.49160350843613565 0.4935589282851221 0.49525991588553131 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400778 0.49788827395673624 0.49670379107443569 0.49525991588553137 0.4935589282851221]
  [0.043207521739130436 -0.019292478260869564 0.45036993447567875 0.48968620869942392 0.46318489954803271 0.4672478131307139 0.47109694804702379 0.47472645280913361 0.47813077932351289 0.48130469701492223 0.48424330621240924 0.48694205074937669 0.48939672973238352 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569 0.49525991588553137]
  [0.0453814347826087 -0.0171185652173913 0.44553902740550355 0.49186174887081541 0.45891434736941544 0.46318489954803266 0.4672478131307139 0.47109694804702379 0.47472645280913361 0.47813077932351283 0.48130469701492223 0.48424330621240924 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400784 0.49788827395673624 0.49670379107443569]
  [0.047555347826086954 -0.014944652173913048 0.44052670709878128 0.49378552477394472 0.45444256980095132 0.45891434736941544 0.46318489954803271 0.4672478131307139 0.47109694804702379 0.47472645280913361 0.47813077932351289 0.48130469701492229 0.48424330621240924 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400778 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662163 0.49947197949314981 0.49881149090400778 0.49788827395673624]
  [0.049729260869565216 -0.012770739130434786 0.43534028926678386 0.495454511749161 0.44977623715650628 0.45444256980095132 0.45891434736941544 0.46318489954803271 0.4672478131307139 0.47109694804702379 0.47472645280913361 0.47813077932351289 0.48130469701492223 0.48424330621240924 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662163 0.500001 0.49986869236662157 0.49947197949314975 0.49881149090400778]
  [0.051903173913043478 -0.010596826086956524 0.42998728018660409 0.49686607930323784 0.44492226030491427 0.44977623715650628 0.45444256980095132 0.45891434736941544 0.46318489954803271 0.4672478131307139 0.47109694804702379 0.47472645280913361 0.47813077932351289 0.48130469701492229 0.48424330621240924 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553131 0.49670379107443569 0.49788827395673624 0.49881149090400784 0.49947197949314981 0.49986869236662157 0.500001 0.49986869236662157 0.49947197949314981]
  [0.05407708695652174 -0.0084229130434782615 0.4244753586656535 0.49801799801789431 0.43988777364963222 0.44492226030491427 0.44977623715650628 0.45444256980095132 0.45891434736941544 0.46318489954803266 0.46724781313071395 0.47109694804702379 0.47472645280913361 0.47813077932351289 0.48130469701492223 0.48424330621240924 0.48694205074937669 0.48939672973238346 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400778 0.49947197949314975 0.49986869236662157 0.500001 0.49986869236662163]
  [0.056251 -0.0062489999999999985 0.41881235776953657 0.49890844541724522 0.4346801176994029 0.43988777364963222 0.44492226030491427 0.44977623715650628 0.45444256980095132 0.45891434736941544 0.46318489954803271 0.46724781313071395 0.47109694804702379 0.47472645280913361 0.47813077932351283 0.48130469701492223 0.48424330621240924 0.48694205074937669 0.48939672973238352 0.49160350843613565 0.4935589282851221 0.49525991588553137 0.49670379107443569 0.49788827395673624 0.49881149090400778 0.49947197949314981 0.49986869236662163 0.500001]